### import packages

In [ ]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity

### Specify directories to data files

In [ ]:
r1_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\"
r2_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\"

# lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
# bad_roi='/test/bad_roi_list.npy'
# spot_dir = '/test/LHA3_R3_medium/spots/'
# out_dir = '/test/LHA3_R3_medium/spots/'
# img_dir='/test/LHA3_R3_medium/stitching/export.n5'


fix_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\export_sigma3.n5"
reg_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped"  # directory to the registered image
out_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped\\testout-july29"  # where the output files should be saved 
lb_dir  = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\segmentation\\lt171_gene_4tile_r2-c3.tif"  # directory to the segmentation mask (tif format accepted here)

bad_roi = os.path.join(out_dir, "bad_roi_list.npy")
spot_dir = os.path.join(r2_dir, "spots")
out_dir = spot_dir
img_dir = fix_dir

print(bad_roi)
print(spot_dir)

### Load segmentation mask

In [ ]:
%%time
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

### Extract ROI properties
The roi_prop function exports the following: 
- ROI ID
- position (x, y, z in um, pre-expansion)
- size (volume, pre-expansion) 
- mionr_axis_length
- major_axis_length
- eccentricity =  mionr_axis_length/major_axis_length
- solidity = ROI volume/ROI convex hull

In [ ]:
%%time

# this takes ~ 1hr
df=roi_prop.roi_prop(lb)

### Remove bad ROIs and save the file

In [ ]:
%%time
df_filtered=df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi=df_filtered.roi.astype(int)
df_filtered.to_csv(os.path.join(out_dir, 'roi.csv'))

In [ ]:
print(out_dir)

### Lipofuscin subtraction (optional)

In [ ]:
%%time
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file

In [ ]:
%%time
spotcount=spot.spot_counts(lb,spot_dir)
spotcount.to_csv(out_dir+'/spots.csv')

### Intensity measurement and save file

In [ ]:
%%time
df_intensity=intensity.measure_intensity(lb, img_dir,'c0')
df_intensity.to_csv(out_dir+'/intennsity_c0.csv')

In [ ]:
df_intensity